# 数据挖掘第六章作业 第7题

## 读入数据

In [122]:
import pandas as pd
def load_data(path):
        data = pd.read_csv(path)
        num = data.iloc[:, -2]
        y = data.iloc[:, -1]

        return num, y

# 生成购物篮，每个订单作为一个集合
def getShoppingCart(orders, items):
    numOfOrder = len(set(orders))
    shoppingCart = [set() for _ in range(numOfOrder)]
    for j in range(len(orders)):
        shoppingCart[orders[j]-1].add(items[j])
    return shoppingCart

orders, items = load_data('data100.csv')
shoppingCart = getShoppingCart(orders, items)
print(shoppingCart)

[{'sandwich bags', 'dinner rolls', 'aluminum foil', 'soda', 'ice cream', 'flour', 'lunch meat', 'butter', 'pork', 'mixes', 'laundry detergent', 'beef', 'shampoo', 'soap', 'all- purpose', 'vegetables'}, {'sandwich bags', 'tortillas', 'waffles', 'cereals', 'aluminum foil', 'cheeses', 'dishwashing liquid/detergent', 'yogurt', 'milk', 'laundry detergent', 'shampoo', 'toilet paper', 'individual meals', 'mixes', 'vegetables', 'hand soap'}, {'cereals', 'spaghetti sauce', 'lunch meat', 'pork', 'laundry detergent', 'shampoo', 'sandwich loaves', 'soda', 'bagels', 'cheeses', 'milk', 'vegetables', 'hand soap', 'dinner rolls', 'ketchup', 'soap', 'ice cream', 'poultry', 'eggs', 'toilet paper'}, {'cereals', 'soda', 'lunch meat', 'all- purpose', 'juice', 'toilet paper'}, {'pasta', 'flour', 'spaghetti sauce', 'pork', 'individual meals', 'sandwich loaves', 'all- purpose', 'soda', 'yogurt', 'milk', 'mixes', 'vegetables', 'hand soap', 'dinner rolls', 'tortillas', 'waffles', 'poultry', 'eggs', 'paper towel

## Apriori算法

### Apriori 算法的实现

In [123]:
FREQUENT = 30


class Apriori:

    # 判断itemList是否在order中
    def __contains(self, order, itemList):
        for item in itemList:
            if item not in order:
                return False
        return True

    # 计算itemList在shoppingCart中出现的次数
    def __count(self, itemList, shoppingCart):
        count = 0
        for order in shoppingCart:
            if (self.__contains(order, itemList)):
                count += 1
        return count

    # 生成下一层的itemList
    def __generate_next_itemList(self, itemList):
        next_itemList = []
        for i in range(len(itemList)):
            for j in range(i+1, len(itemList)):
                if itemList[i][:-1] == itemList[j][:-1]:
                    newList = [x for x in itemList[i]]
                    newList.append(itemList[j][-1])
                    next_itemList.append(newList)
        return next_itemList

    # 生成第一层的itemList
    def __getFirstItemLists(self, items):
        firstItemList = [[x] for x in set(items)]
        return firstItemList

    # 过滤掉不满足最小支持度的itemList
    def __filter(self, itemLists, shoppingCart):
        filterList = []
        for itemList in itemLists:
            if self.__count(itemList, shoppingCart) >= FREQUENT:
                filterList.append(itemList)
        return filterList

    # 主函数,也是对外接口
    def analyze(self, items, shoppingCart):
        itemLists = self.__getFirstItemLists(items)
        totalFrequentItemLists = []

        while len(itemLists) > 0:
            filteredItemLists = []
            filteredItemLists = self.__filter(itemLists, shoppingCart)
            if (len(filteredItemLists) > 0):
                totalFrequentItemLists.append(filteredItemLists)
            nextItemList = self.__generate_next_itemList(filteredItemLists)
            itemLists = nextItemList

        return totalFrequentItemLists


### Apriori算法的使用

In [124]:
# 输出所有的频繁项集
ap = Apriori()
print(ap.analyze(items, shoppingCart))

[[['coffee/tea'], ['sandwich bags'], ['cereals'], ['pasta'], ['flour'], ['lunch meat'], ['spaghetti sauce'], ['pork'], ['laundry detergent'], ['shampoo'], ['individual meals'], ['sandwich loaves'], ['all- purpose'], ['soda'], ['cheeses'], ['bagels'], ['dishwashing liquid/detergent'], ['yogurt'], ['milk'], ['mixes'], ['vegetables'], ['hand soap'], ['dinner rolls'], ['aluminum foil'], ['butter'], ['beef'], ['ketchup'], ['soap'], ['juice'], ['sugar'], ['tortillas'], ['waffles'], ['ice cream'], ['poultry'], ['eggs'], ['paper towels'], ['fruits'], ['toilet paper']], [['coffee/tea', 'vegetables'], ['lunch meat', 'vegetables'], ['shampoo', 'vegetables'], ['sandwich loaves', 'vegetables'], ['all- purpose', 'vegetables'], ['soda', 'vegetables'], ['cheeses', 'vegetables'], ['vegetables', 'aluminum foil'], ['vegetables', 'beef'], ['vegetables', 'ketchup'], ['vegetables', 'waffles'], ['vegetables', 'ice cream'], ['vegetables', 'poultry']]]


## FP-growth算法

### FP-growth 算法的实现

In [125]:
FREQUENT = 30


class FPNode:
    # basic node in FP tree
    def __init__(self, name, count, parent):
        self.name = name
        self.count = count
        self.parent = parent
        self.children = {}
        self.sameNodeLink = None

    # add count to node
    def more(self, count):
        self.count += count


class FPAlgorithm:
    def __init__(self, minSup):
        self.minSup = minSup

    # 将原始事务集转换为字典形式

    def createInitSet(self, dataSet):
        retDict = {}
        for trans in dataSet:
            retDict[frozenset(trans)] = 1
        return retDict

    # 同一个元素的链表，尾插法
    def updateHeader(self, nodeToTest, targetNode):
        while (nodeToTest.sameNodeLink is not None):
            nodeToTest = nodeToTest.sameNodeLink
        nodeToTest.sameNodeLink = targetNode

    # 追踪一条路径
    def ascendTree(self, leafNode, prefixPath):
        if leafNode.parent is not None:
            prefixPath.append(leafNode.name)
            self.ascendTree(leafNode.parent, prefixPath)

    def findPrefixPath(self, basePat, treeNode):
        condPats = {}
        while treeNode is not None:
            prefixPath = []
            self.ascendTree(treeNode, prefixPath)
            if len(prefixPath) > 1:
                condPats[frozenset(prefixPath[1:])] = treeNode.count
            treeNode = treeNode.sameNodeLink
        return condPats

    def updateTree(self, items, inTree, headerTable, count):
        if items[0] in inTree.children:
            inTree.children[items[0]].more(count)
        else:
            inTree.children[items[0]] = FPNode(items[0], count, inTree)
            if headerTable[items[0]][1] is None:
                headerTable[items[0]][1] = inTree.children[items[0]]
            else:
                self.updateHeader(headerTable[items[0]][1],
                                  inTree.children[items[0]])

        if len(items) > 1:
            self.updateTree(items[1:], inTree.children[items[0]],
                            headerTable, count)

    def createTree(self, dataset):
        headerTable = {}
        for order in dataset:
            for item in order:
                headerTable[item] = headerTable.get(item, 0) + 1

        for k in list(headerTable.keys()):
            if headerTable[k] < FREQUENT:
                del (headerTable[k])

        freqItemSet = set(headerTable.keys())
        if len(freqItemSet) == 0:
            return None, None

        for k in headerTable:
            headerTable[k] = [headerTable[k], None]

        root = FPNode('Null Set', 1, None)

        for tran, count in dataset.items():
            localD = {}
            for item in tran:
                if item in freqItemSet:
                    localD[item] = headerTable[item][0]
            if len(localD) > 0:
                orderedItems = [v[0]
                                for v in sorted(localD.items(), key=lambda p:p[1], reverse=True)]
                self.updateTree(orderedItems, root, headerTable, count)

        return root, headerTable

    def mineTree(self, inTree, headerTable, minSup, preFix, freqItemList):
        bigL = [v[0]
                for v in sorted(headerTable.items(), key=lambda p:str(p[1]))]
        for basePat in bigL:
            newFreqSet = preFix.copy()
            newFreqSet.add(basePat)

            freqItemList.append(newFreqSet)
            condPattBases = self.findPrefixPath(
                basePat, headerTable[basePat][1])
            myCondTree, myHead = self.createTree(condPattBases)
            if myHead is not None:
                self.mineTree(myCondTree, myHead, minSup,
                              newFreqSet, freqItemList)

    def analyse(self, shoppingCart):
        self.trans = shoppingCart
        self.dataset = self.createInitSet(shoppingCart)
        root, headerTable = self.createTree(self.dataset)
        freqItemList = []
        self.mineTree(root, headerTable, FREQUENT, set([]), freqItemList)
        return freqItemList


### FP-growth 算法的使用

In [126]:
fp = FPAlgorithm(FREQUENT)
print(fp.analyse(shoppingCart))

[{'yogurt'}, {'flour'}, {'soap'}, {'dinner rolls'}, {'laundry detergent'}, {'sandwich bags'}, {'milk'}, {'butter'}, {'eggs'}, {'dishwashing liquid/detergent'}, {'tortillas'}, {'pork'}, {'spaghetti sauce'}, {'coffee/tea'}, {'coffee/tea', 'vegetables'}, {'pasta'}, {'sugar'}, {'waffles'}, {'waffles', 'vegetables'}, {'hand soap'}, {'aluminum foil'}, {'vegetables', 'aluminum foil'}, {'paper towels'}, {'all- purpose'}, {'all- purpose', 'vegetables'}, {'cereals'}, {'mixes'}, {'ice cream'}, {'vegetables', 'ice cream'}, {'lunch meat'}, {'lunch meat', 'vegetables'}, {'individual meals'}, {'toilet paper'}, {'bagels'}, {'fruits'}, {'sandwich loaves'}, {'beef'}, {'ketchup'}, {'shampoo'}, {'juice'}, {'poultry'}, {'soda'}, {'cheeses'}, {'vegetables'}]


## ECLAT算法

### ECLAT算法的实现

In [127]:
FREQUENT = 30


class Eclat:

    def __init__(self, min_support=FREQUENT, max_items=2):
        self.min_support = min_support
        self.max_items = max_items
        self.result = []
        self.itemDict = {}

    def fit(self, transactions):
        self.transactions = transactions
        self.num_transactions = len(transactions)
        self._get_items()

    def _get_items(self):
        items = []
        k = 0
        for transaction in self.transactions:
            k += 1
            for item in transaction:
                if([item] not in items):
                    items.append([item])
                if(frozenset([item]) not in self.itemDict):
                    self.itemDict[frozenset([item])] = set()
                self.itemDict[frozenset([item])].add(k)
        self.items = sorted(list(items))
        self.filter(self.itemDict, self.items)

    def filter(self, itemDict, keys):
        for key in keys:
            if(len(itemDict[frozenset(key)]) < self.min_support):
                del itemDict[frozenset(key)]
            else:
                self.result.append(key)
        return itemDict

    def _get_frequent_itemsets(self):
        nextRound = self.items
        for i in range(0, self.max_items):
            nextRound = self.__generate_next_itemList(nextRound)
            self.itemDict = self.filter(self.itemDict, nextRound)
            if not nextRound:
                break 
        return self.result
        
    def __generate_next_itemList(self, itemList):
        next_itemList = []
        for i in range(len(itemList)):
            for j in range(i+1, len(itemList)):
                if ((itemList[i][:-1] == itemList[j][:-1]) and (self.itemDict.get(frozenset(itemList[i]), {})) and (self.itemDict.get(frozenset(itemList[j]), {}))):
                    newList = [x for x in itemList[i]]
                    newList.append(itemList[j][-1])
                    next_itemList.append(newList)
                    self.itemDict[frozenset(newList)] = self.itemDict[frozenset(itemList[i])].intersection(self.itemDict[frozenset(itemList[j])])
        return next_itemList
    


### ECLAT算法的使用

In [128]:
def get_frequent_itemsets(shoppingCart):
    transactions = shoppingCart
    eclat = Eclat(min_support=FREQUENT, max_items=2)
    eclat.fit(transactions)
    frequent_itemsets = eclat._get_frequent_itemsets()
    print("final result:", frequent_itemsets)


get_frequent_itemsets(shoppingCart)

final result: [['all- purpose'], ['aluminum foil'], ['bagels'], ['beef'], ['butter'], ['cereals'], ['cheeses'], ['coffee/tea'], ['dinner rolls'], ['dishwashing liquid/detergent'], ['eggs'], ['flour'], ['fruits'], ['hand soap'], ['ice cream'], ['individual meals'], ['juice'], ['ketchup'], ['laundry detergent'], ['lunch meat'], ['milk'], ['mixes'], ['paper towels'], ['pasta'], ['pork'], ['poultry'], ['sandwich bags'], ['sandwich loaves'], ['shampoo'], ['soap'], ['soda'], ['spaghetti sauce'], ['sugar'], ['toilet paper'], ['tortillas'], ['vegetables'], ['waffles'], ['yogurt'], ['all- purpose', 'vegetables'], ['aluminum foil', 'vegetables'], ['beef', 'vegetables'], ['cheeses', 'vegetables'], ['coffee/tea', 'vegetables'], ['ice cream', 'vegetables'], ['ketchup', 'vegetables'], ['lunch meat', 'vegetables'], ['poultry', 'vegetables'], ['sandwich loaves', 'vegetables'], ['shampoo', 'vegetables'], ['soda', 'vegetables'], ['vegetables', 'waffles']]


## 不同数据集上运行时间的比较实验

### 100个事务

In [129]:
import time

# on 100 trans

start = time.time()
ap = Apriori()
print(ap.analyze(items, shoppingCart))
end = time.time()
print("Apriori time on 100 trans:", (end - start) * 1000)

start = time.time()
fp = FPAlgorithm(FREQUENT)
print(fp.analyse(shoppingCart))
end = time.time()
print("FP-growth time on 100 trans:", (end - start) * 1000)

start = time.time()
get_frequent_itemsets(shoppingCart)
end = time.time()
print("ECLAT time on 100 trans:", (end - start) * 1000)



[[['coffee/tea'], ['sandwich bags'], ['cereals'], ['pasta'], ['flour'], ['lunch meat'], ['spaghetti sauce'], ['pork'], ['laundry detergent'], ['shampoo'], ['individual meals'], ['sandwich loaves'], ['all- purpose'], ['soda'], ['cheeses'], ['bagels'], ['dishwashing liquid/detergent'], ['yogurt'], ['milk'], ['mixes'], ['vegetables'], ['hand soap'], ['dinner rolls'], ['aluminum foil'], ['butter'], ['beef'], ['ketchup'], ['soap'], ['juice'], ['sugar'], ['tortillas'], ['waffles'], ['ice cream'], ['poultry'], ['eggs'], ['paper towels'], ['fruits'], ['toilet paper']], [['coffee/tea', 'vegetables'], ['lunch meat', 'vegetables'], ['shampoo', 'vegetables'], ['sandwich loaves', 'vegetables'], ['all- purpose', 'vegetables'], ['soda', 'vegetables'], ['cheeses', 'vegetables'], ['vegetables', 'aluminum foil'], ['vegetables', 'beef'], ['vegetables', 'ketchup'], ['vegetables', 'waffles'], ['vegetables', 'ice cream'], ['vegetables', 'poultry']]]
Apriori time on 100 trans: 10.969161987304688
[{'yogurt'},

### 300个事务

In [130]:
import time

# on 100 trans
orders, items = load_data('data300.csv')
bigCart = getShoppingCart(orders, items)

start = time.time()
ap = Apriori()
print(ap.analyze(items, bigCart))
end = time.time()
print("Apriori time on 300 trans:", (end - start) * 1000)

start = time.time()
fp = FPAlgorithm(FREQUENT)
print(fp.analyse(bigCart))
end = time.time()
print("FP-growth time on 300 trans:", (end - start) * 1000)

start = time.time()
get_frequent_itemsets(bigCart)
end = time.time()
print("ECLAT time on 300 trans:", (end - start) * 1000)


[[['coffee/tea'], ['sandwich bags'], ['cereals'], ['pasta'], ['flour'], ['lunch meat'], ['spaghetti sauce'], ['pork'], ['laundry detergent'], ['shampoo'], ['individual meals'], ['sandwich loaves'], ['all- purpose'], ['soda'], ['cheeses'], ['bagels'], ['dishwashing liquid/detergent'], ['yogurt'], ['milk'], ['mixes'], ['vegetables'], ['hand soap'], ['dinner rolls'], ['aluminum foil'], ['butter'], ['beef'], ['ketchup'], ['soap'], ['juice'], ['sugar'], ['tortillas'], ['waffles'], ['ice cream'], ['poultry'], ['eggs'], ['paper towels'], ['fruits'], ['toilet paper']], [['coffee/tea', 'sandwich bags'], ['coffee/tea', 'cereals'], ['coffee/tea', 'pasta'], ['coffee/tea', 'flour'], ['coffee/tea', 'lunch meat'], ['coffee/tea', 'spaghetti sauce'], ['coffee/tea', 'pork'], ['coffee/tea', 'laundry detergent'], ['coffee/tea', 'shampoo'], ['coffee/tea', 'individual meals'], ['coffee/tea', 'sandwich loaves'], ['coffee/tea', 'all- purpose'], ['coffee/tea', 'soda'], ['coffee/tea', 'cheeses'], ['coffee/tea',

### 上面的实验结果是相当显著的，可以看出在大数据集上，ECLAT算法的效率是最高的，其次是FP-growth算法，最后是Apriori算法。然而在小数据集上，ECLAT落后于另外两者。这是因为ECLAT常数开销大，但复杂度低，而Apriori常数开销小，但复杂度高。而FP-growth算法介于两者之间。

## 三种算法的原理与性能比较


1. Apriori基于连接，它自底向上，通过多次扫描数据库并应用最小支持度阈值来生成频繁项集。但是它的时间和内存成本开销比较大。

1. FP-tree基于树，它使用将数据库压缩为FP-tree，保留了项集关联信息。不需要再次扫描数据库。它更快更高效，特别是对于稀疏数据集。然而，它仍然可能生成大量的条件FP-tree，消耗内存。

1. ECLAT基于模式增长，它适用于密集数据集或低最小支持度阈值。然而，它可能在处理高维数据集时，造成高内存消耗。

这些算法的性能取决于各种因素，如数据大小，数据分布，最小支持度阈值和模式密度等等，在某些情形下一种算法可能好于另外的算法。
- Apriori适用于小型或稀疏数据集，具有高最小支持度阈值和低模式密度。

- FP-tree适用于大型或稀疏数据集，具有中等最小支持度阈值和中等模式密度。

- ECLAT适用于大型或密集数据集，具有低最小支持度阈值和高模式密度。
